In [2]:
from gensim.models import KeyedVectors, Word2Vec
from gensim.parsing.preprocessing import remove_stopwords
from tinydb import TinyDB, Query
import numpy as np
import datetime
import utils
import spacy
import re
%load_ext autoreload
%autoreload 2

In [3]:
# Load the polish Word2Vec model
word2vec = KeyedVectors.load('./word2vec_100_3_polish.bin')

In [9]:
db_gov_news = TinyDB('./data/gov_news.json')
db_gov_announcements = TinyDB('./data/gov_announcements.json')

news = {
    'Wyborcza': TinyDB('./data/wyborcza.json'),
    'TVP Info': TinyDB('./data/tvpinfo.json'),
    'Polsat': TinyDB('./data/polsat.json'),
    'NaTemat': TinyDB('./data/natemat.json'),
    'Niezalezna': TinyDB('./data/niezalezna.json'),
    'Onet': TinyDB('./data/onet.json'),
    'TVN24': TinyDB('./data/tvn24.json'),
    'Rzeczpospolita': TinyDB('./data/rp.json'),
}

In [5]:
q = Query()

In [6]:
nlp = spacy.load('pl_core_news_lg')
stop_words = ['a', 'b', 'c', 'ć', 'd', 'e', 'ę', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'ł', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'w', 'y', 'z', 'ź', 'ż' 'aby', 'ach', 'acz', 'aczkolwiek', 'aj', 'albo', 'ale', 'alez', 'ależ', 'ani', 'az', 'aż', 'bardziej', 'bardzo', 'beda', 'bedzie', 'bez', 'deda', 'będą', 'bede', 'będę', 'będzie', 'bo', 'bowiem', 'by', 'byc', 'być', 'byl', 'byla', 'byli', 'bylo', 'byly', 'był', 'była', 'było', 'były', 'bynajmniej', 'cala', 'cali', 'caly', 'cała', 'cały', 'ci', 'cie', 'ciebie', 'cię', 'co', 'cokolwiek', 'cos', 'coś', 'czasami', 'czasem', 'czemu', 'czy', 'czyli', 'daleko', 'dla', 'dlaczego', 'dlatego', 'do', 'dobrze', 'dokad', 'dokąd', 'dosc', 'dość', 'duzo', 'dużo', 'dwa', 'dwaj', 'dwie', 'dwoje', 'dzis', 'dzisiaj', 'dziś', 'gdy', 'gdyby', 'gdyz', 'gdyż', 'gdzie', 'gdziekolwiek', 'gdzies', 'gdzieś', 'go', 'i', 'ich', 'ile', 'im', 'inna', 'inne', 'inny', 'innych', 'iz', 'iż', 'ja', 'jak', 'jakas', 'jakaś', 'jakby', 'jaki', 'jakichs', 'jakichś', 'jakie', 'jakis', 'jakiś', 'jakiz', 'jakiż', 'jakkolwiek', 'jako', 'jakos', 'jakoś', 'ją', 'je', 'jeden', 'jedna', 'jednak', 'jednakze', 'jednakże', 'jedno', 'jego', 'jej', 'jemu', 'jesli', 'jest', 'jestem', 'jeszcze', 'jeśli', 'jezeli', 'jeżeli', 'juz', 'już', 'kazdy', 'każdy', 'kiedy', 'kilka', 'kims', 'kimś', 'kto', 'ktokolwiek', 'ktora', 'ktore', 'ktorego', 'ktorej', 'ktory', 'ktorych', 'ktorym', 'ktorzy', 'ktos', 'ktoś', 'która', 'które', 'którego', 'której', 'który', 'których', 'którym', 'którzy', 'ku', 'lat', 'lecz', 'lub', 'ma', 'mają', 'mało', 'mam', 'mi', 'miedzy', 'między', 'mimo', 'mna', 'mną', 'mnie', 'moga', 'mogą', 'moi', 'moim', 'moj', 'moja', 'moje', 'moze', 'mozliwe', 'mozna', 'może', 'możliwe', 'można', 'mój', 'mu', 'musi', 'my', 'na', 'nad', 'nam', 'nami', 'nas', 'nasi', 'nasz', 'nasza', 'nasze', 'naszego', 'naszych', 'natomiast', 'natychmiast', 'nawet', 'nia', 'nią', 'nic', 'nich', 'nie', 'niech', 'niego', 'niej', 'niemu', 'nigdy', 'nim', 'nimi', 'niz', 'niż', 'no', 'o', 'obok', 'od', 'około', 'on', 'ona', 'one', 'oni', 'ono', 'oraz', 'oto', 'owszem', 'pan', 'pana', 'pani', 'po', 'pod', 'podczas', 'pomimo', 'ponad', 'poniewaz', 'ponieważ', 'powinien', 'powinna', 'powinni', 'powinno', 'poza', 'prawie', 'przeciez', 'przecież', 'przed', 'przede', 'przedtem', 'przez', 'przy', 'roku', 'rowniez', 'również', 'sam', 'sama', 'są', 'sie', 'się', 'skad', 'skąd', 'soba', 'sobą', 'sobie', 'sposob', 'sposób', 'swoje', 'ta', 'tak', 'taka', 'taki', 'takie', 'takze', 'także', 'tam', 'te', 'tego', 'tej', 'ten', 'teraz', 'też', 'to', 'toba', 'tobą', 'tobie', 'totez', 'toteż', 'totobą', 'trzeba', 'tu', 'tutaj', 'twoi', 'twoim', 'twoj', 'twoja', 'twoje', 'twój', 'twym', 'ty', 'tych', 'tylko', 'tym', 'u', 'w', 'wam', 'wami', 'was', 'wasz', 'wasza', 'wasze', 'we', 'według', 'wiele', 'wielu', 'więc', 'więcej', 'wlasnie', 'właśnie', 'wszyscy', 'wszystkich', 'wszystkie', 'wszystkim', 'wszystko', 'wtedy', 'wy', 'z', 'za', 'zaden', 'zadna', 'zadne', 'zadnych', 'zapewne', 'zawsze', 'ze', 'zeby', 'zeznowu', 'zł', 'znow', 'znowu', 'znów', 'zostal', 'został', 'żaden', 'żadna', 'żadne', 'żadnych', 'że', 'żeby']

# Output format: list of lists containing preprocessed words for each sentence in the text
def preprocess(text):
    sentences = []

    for sentence in re.split('[//.|//!|//?]', text):
        if sentence != '':
            sentence = re.sub(r'[^A-Za-ząćęłńóśźżĄĆĘŁŃÓŚŹŻ ]+', '', sentence).lower().strip().split()
            sentence_tokens = [word for word in sentence if word.strip() not in stop_words]
            sentences.append(sentence_tokens)
    
    return sentences

# Compute the mean computationally for each sentence in the text
def compute_mean(sentence):
    mean_v = np.zeros((100,))
    skipped = []
    for i in range(len(sentence)):
        word = sentence[i]
        word_lemma = nlp(word)[0].lemma_.lower()

        try:
            mean_v += (word2vec[word_lemma] - mean_v) / (i+1)
            #print(word_lemma)
        except KeyError:
            skipped.append(word)

    return mean_v

In [11]:
for news_provider in news:

    textual_reuse_count = 0
    print('Analyzing %s...' % news_provider)

    for article in news[news_provider]:
        processed_article = utils.preprocess(article['content'])
        article_date = datetime.datetime.fromtimestamp(article['release_date'])

        # Find source articles from around the same time
        gov_news = db_gov_news.search(
            (q.release_date > (article_date - datetime.timedelta(days=2)).timestamp())
            &
            (q.release_date < article['release_date'])
        )
        gov_announcements = db_gov_announcements.search(
            (q.release_date > (article_date - datetime.timedelta(days=2)).timestamp())
            &
            (q.release_date < article['release_date'])
        )

        if len(gov_news) == 0 and len(gov_announcements) == 0:
            continue

        for sentence in processed_article:
            if len(sentence) < 4:
                continue

            sentence_mean = compute_mean(sentence)

            # For every source article
            for source in gov_news+gov_announcements:
                processed_source = utils.preprocess(source['content'])

                for source_sentence in processed_source:
                    if len(source_sentence) < 4:
                        continue

                    source_sentence_mean = compute_mean(source_sentence)

                    cosine_sim = np.dot(sentence_mean, source_sentence_mean) / (np.linalg.norm(sentence_mean)*np.linalg.norm(source_sentence_mean))
                    if cosine_sim>0.9:
                        textual_reuse_count += 1
                        print()
                        print('Textual reuse detected!')
                        print(sentence, source_sentence)
                        print(article)
                        print(source)
                        print('Cosine similarity: %s' % cosine_sim)
              
    print('News provider: %s, Textual reuse count: %s' % (news_provider, textual_reuse_count))
                

Analyzing Wyborcza...

Textual reuse detected!
['sąd', 'okręgowy', 'podtrzymał', 'ocenę', 'sądu', 'rejonowego', 'kublik', 'opublikowanych', 'artykułach', 'prasowych', 'wyrażała', 'zaniepokojenie', 'możliwym', 'sposobem', 'działania', 'niektórych', 'osób', 'kręgu', 'obecnej', 'władzy', 'obraz', 'wyłaniał', 'informacji', 'ujawnionych', 'andrzeja', 'malinowskiego'] ['nieprowadzeniu', 'wobec', 'postępowania', 'przestępstwo', 'ścigane', 'oskarżenia', 'publicznego', 'przestępstwo', 'skarbowew', 'zakresie', 'przynależności', 'partii', 'politycznych', 'pełnienia', 'funkcji', 'publicznych', 'oświadczenie', 'zapoznaniu', 'treścią', 'klauzuli', 'informacyjnej', 'kandydata', 'stanowisko', 'dyrektora', 'sądu', 'okręgowego', 'lublinie', 'wzór', 'załączeniuoświadczenie', 'wyrażeniu', 'zgody', 'przetwarzanie', 'danych', 'osobowych', 'kandydatawzór', 'załączeniu']
{'title': 'Woyciechowski przegrał z Agnieszką Kublik. Sąd: "Jej relacja była niewątpliwie rzetelna"', 'url': 'https://wyborcza.pl/7,75398,30